In [1]:
import tensorflow as tf

In [2]:
a = tf.constant(5,name="input_a")
b = tf.constant(3,name="input_b")
c = tf.multiply(a,b,name="mul_c")
d = tf.add(a,b,name="add_d")
e = tf.add(c,d,name="add_e")
sess = tf.Session()
sess.run(e)

23

In [3]:
output = sess.run(e)

In [4]:
writer = tf.summary.FileWriter('C:/Users/SKJ/Documents/GitHub/graph',sess.graph)

In [5]:
writer.close()
sess.close()

In [6]:
#换成张量输入
a = tf.constant([5,3],name="input_a")
b = tf.reduce_prod(a,name="prod_b")
c = tf.reduce_sum(a,name="sum_c")
d = tf.add(c,d,name="add_d")

In [7]:
import numpy as np
t_0 = np.array(50,dtype=np.int32)#养成声明dtype的习惯
t_1 = np.array([b"apple",b"peach",b"grape"])#创建字符串类型时不能声明dtype
t_2 = np.array([[True,False,False],[False,False,True],
                [False,True,False]],dtype = np.bool)
t_3 =  np.array([[[0,0],[0,1],[0,2]],
                 [[1,0],[1,1],[1,2]],
                 [[2,0],[2,1],[2,2]]],dtype = np.int64)

In [36]:
#通过名称作用域组织数据流图
import tensorflow as tf
graph = tf.Graph()
with graph.as_default():
    in_1 = tf.placeholder(tf.float32,shape=[],name="input_a")
    in_2 = tf.placeholder(tf.float32,shape=[],name="input_b")
    const = tf.constant(3,dtype=tf.float32,name="static_value")
    with tf.name_scope("A"):
        A_mul = tf.multiply(in_1,const)
        A_out = tf.subtract(A_mul,in_1)
    with tf.name_scope("B"):
        B_mul = tf.multiply(in_2,const)
        B_out = tf.subtract(B_mul,in_2)
    with tf.name_scope("C"):
        C_div = tf.div(A_out,B_out)
        C_out = tf.add(C_div,const)
    with tf.name_scope("D"):
        D_div = tf.div(B_out,A_out)
        D_out = tf.add(D_div,const) 
    writer = tf.summary.FileWriter('C:/Users/SKJ/Documents/GitHub/graph',graph=graph)
    writer.close()  

In [63]:
#练习组件
#-*- coding: utf-8 -*-
import tensorflow as tf
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope("variables"):
        global_step = tf.Variable(0,dtype = tf.int32,\
                                  trainable = False,name = "global_step")
        total_output = tf.Variable(0.0,dtype = tf.float32,\
                                   trainable = False,name = "total_output")
    with tf.name_scope("transformation"):
        #独立的输入层
        with tf.name_scope("input"):
            a = tf.placeholder(tf.float32,shape=[None],name="input_placeholder_a")
        #独立的中间层
        with tf.name_scope("intermediate_layer"):
            b = tf.reduce_prod(a,name="product_b")
            c = tf.reduce_sum(a,name="sum_c")
        #独立的输出层
        with tf.name_scope("out_put"):
            output = tf.add(b,c,name="out_put")
    with tf.name_scope("update"):
        update_total = total_output.assign_add(output)
        increment_step = global_step.assign_add(1)
    with tf.name_scope("summaries"):
        avg = tf.div(update_total,tf.cast(increment_step,tf.float32),name="average")
        tf.summary.scalar('Output',output)#name="output_summary"
        tf.summary.scalar('Sum_of_outputs_over_time',update_total)#,name="total_summary")
        tf.summary.scalar('Average_of_outputs_over_time',avg)#,name="average_summary")
    with tf.name_scope("global_ops"):
        init = tf.global_variables_initializer()
        merged_summaries = tf.summary.merge_all() 
        
sess = tf.Session(graph=graph)
writer = tf.summary.FileWriter('C:/Users/SKJ/Documents/GitHub/graph',graph=graph)
sess.run(init)

def run_graph(input_tensor):
    feed_dict = {a:input_tensor}
    _,step,summary = sess.run([output,increment_step,merged_summaries],\
                              feed_dict = feed_dict)
    writer.add_summary(summary,global_step = step)

run_graph([2,8])
run_graph([3,1,3,3])
run_graph([8])
run_graph([1,2,3])
run_graph([11,4])
run_graph([4,1])
run_graph([7,3,1])
run_graph([6,3])
run_graph([0,2])
run_graph([4,5,6])

writer.flush()
writer.close()
sess.close()